In [7]:
import json
from pathlib import Path
import pickle

import datasets
import numpy as np
import torch
import tqdm
import transformers

In [8]:
DATASET_NAME = "imdb"
RETRIEVER_NAME = "facebook/contriever"
OUTPUT_DIR = Path(f"vectors_{DATASET_NAME}_{RETRIEVER_NAME.split('/')[-1]}")
BATCH_SIZE = 100
DEVICE = 2


OUTPUT_DIR.mkdir(exist_ok=True)
with (OUTPUT_DIR / "config.json").open("w") as f:
    json.dump(dict(dataset_name=DATASET_NAME, retriever_name=RETRIEVER_NAME), f)

In [9]:
dataset = datasets.load_dataset(DATASET_NAME)


Reusing dataset imdb (/home/mila/g/gagnonju/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
samples = [x["text"] for x in dataset["train"]]
labels = [x["label"] for x in dataset["train"]]
with (OUTPUT_DIR / "train_samples.json").open("w") as f:
    json.dump(dict(inputs=samples, labels=labels), f, indent=4)



In [4]:
model = transformers.AutoModel.from_pretrained(RETRIEVER_NAME).to(DEVICE)
tokenizer = transformers.AutoTokenizer.from_pretrained(RETRIEVER_NAME)

In [6]:
def batcher(array, size):
    for i in range(0, len(array), size):
        yield array[i : i + size]

all_representations = []
for batch in tqdm.tqdm(batcher(samples, BATCH_SIZE), total=len(samples) // BATCH_SIZE):
    batch_toked = tokenizer.batch_encode_plus(batch, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    with torch.inference_mode():
        representations = model(**batch_toked).pooler_output.detach().cpu().numpy()
    all_representations.append(representations)

NameError: name 'asdasdas' is not defined

In [ ]:
embeddings = np.concatenate(all_representations, axis=0)
with (OUTPUT_DIR / "train_vectors.npy").open("wb") as f:
    np.save(f, embeddings)